In [2]:
#needed after restructuring of directory
import os
os.chdir('C:\\Users\\Alex\\OneDrive\\Documents\\GitHub\\UFC_Prediction_2022')

#!/usr/bin/env python
# coding: utf-8

# In[ ]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

#turning csv files to json files
import csv
import json
 
# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath, column):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # primary key given by column variable
            key = rows[column]
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))


# Testing 

In [2]:
def get_next_fight_card():
    url = 'https://www.bestfightodds.com'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser") 
    mycards = soup.find_all("div", {"class": "table-div"})
    ufc_cards = [card for card in mycards if 'ufc' in card.find("a",href=lambda x: x and x.startswith('/events')).get_text().lower()]
    card = ufc_cards[0]
    card_date = card.find("span",{"class":"table-header-date"}).get_text()
    card_title = card.find("a",href=lambda x: x and x.startswith('/events')).get_text()
    #bookies = [bookie.get_text() for bookie in card.find_all("a",href=lambda x: x and x.startswith('/out'))]
    fighter_divs = card.find_all('span',{"class":"t-b-fcc"})
    fighters_list = [fighter.get_text() for fighter in fighter_divs]
    if len(fighters_list)%2 == 0 and all(fighters_list[i] == fighters_list[i+26] for i in range(len(fighters_list)//2)):
        print('Usual Scraping Structure Detected')
    else:
        print('Scraping Structure Has Changed... Check HTML at BestFightOdds.com')
    fights_list = [fighters_list[i:i+2] for i in range(0,len(fighters_list),2)]
    print(f'Upcoming card {card_title} on {card_date} has {len(fights_list)} fights')
    for fight in fights_list:
        print(fight)
    return card_date, card_title, fights_list


In [ ]:
card_date, card_title, fights_list = get_next_fight_card()

# Lets make sure we can add this data in a way that doesn't break the existing data regime

Recall how the program works:

On Github Actions once per week we run

```
name: Update Data
    run: |
        cd src
        python3 update_data_csvs_and_jsons.py
        python3 update_and_rebuild_model.py
```

update_data_csvs_and_jsons does the following:
- bring fight_hist.csv up to date (contains stats from each individual fight)
- bring fighter_stats.csv up to date (contains physical stats from each individual fighter)
- bring ufc_fights_crap.csv up to date (this is compiled from fight_hist and includes computed average stats for each fight one can use to predict fights)
- bring ufc_fights.csv up to date (cleaned version of ufc_fights_crap which can actually be used to predict fights)
- convert ufc_fights_crap to json as ufcfightscrap.json to use in website computations
- scrapes pictures of new fighers added to the roster

updata_and_rebuild_model does the following:
- uses new csv files from above to rebuild the prediction model and updates jsons containing model coefficients for website predictions
- populates vegas_odds.json with new fights and their vegas odds (we will rid of the odds feature for now)
- makes predictions for all of the fights in vegas_odds.csv for which it can
- fills in predictions to vegas_odds.csv and concatenates these predictions to prediction_history.json
- saves over prediction_history.json


In [4]:
vegas_odds=pd.read_json('src/content/data/external/vegas_odds.json')
prediction_history = pd.read_json('src/content/data/external/prediction_history.json')

In [ ]:
vegas_odds

In [5]:
prediction_history.head()

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref,predicted fighter odds,predicted opponent odds,average bookie odds,correct?
0,Angela Hill,,145,145,150,144,,+150,150,+150,146,,Mackenzie Dern,,-175,-170,-190,-186,,-190,-175,-188,-171,,305,-305,"[148, -181]",
1,Anthony Hernandez,,-210,-200,-235,-245,,-235,-225,-250,-191,,Edmen Shahbazyan,,170,170,185,186,,185,188,+200,166,,-139,139,"[-224, 181]",
2,Emily Ducote,,130,125,128,128,,+128,130,+120,133,,Loopy Godinez,,-160,-150,-159,-164,,-159,-160,-150,-153,,210,-210,"[128, -157]",
3,Andre Fialho,,185,190,180,172,,+180,188,+175,180,,Joaquin Buckley,,-225,-230,-230,-225,,-230,-225,-225,-210,,157,-157,"[181, -225]",
4,Diego Ferreira,,-155,-150,-167,-156,,-167,-160,-163,-163,,Michael Johnson,,125,125,135,122,,135,130,+130,143,,-144,144,"[-160, 131]",


Goad: Add new fights to prediction_history from `card_date, card_title, fights_list = get_next_fight_card()` instead of from vegas_odds.json

In [ ]:
col_names = list(prediction_history.columns)
col_values = [['' for _ in range(len(fights_list))] for _ in range(len(col_names))]
d = dict(zip(col_names, col_values))
vegas_odds = pd.DataFrame(data=d)

In [ ]:
import datetime
year = datetime.date.today().strftime('%B %d, %Y')[-4:]
year

In [ ]:
# input looks like 'July 15th'. Need to add year to it
def convert_scraped_date_to_standard_date(input_date):
    year = datetime.date.today().strftime('%B %d, %Y')[-4:]
    month = input_date.split(' ')[0]
    day = input_date.split(' ')[1]
    i = 0
    while day[i].isdigit():
        i+=1
    day = day[:i]
    return f'{month} {day}, {year}'

In [ ]:
convert_scraped_date_to_standard_date(card_date)

In [ ]:
vegas_odds['fighter name'].loc[:] = [fight[0] for fight in fights_list]
vegas_odds['opponent name'].loc[:] = [fight[1] for fight in fights_list]
vegas_odds['date'] = convert_scraped_date_to_standard_date(card_date)


In [ ]:
vegas_odds[['fighter name','opponent name','date']]

In [ ]:
#print('scraping bookie odds from bestfightodds.com')
#odds_df = get_odds()
#odds_df=drop_irrelevant_fights(odds_df,3) #allows 3 bookies to have missing odds. can increase this to 2 or 3 as needed
#odds_df=drop_non_ufc_fights(odds_df)
#odds_df=drop_repeats(odds_df)
#print('saving odds to content/data/external/vegas_odds.json')
#save to json
#result = odds_df.to_json()
#parsed = json.loads(result)
#jsonFilePath='content/data/external/vegas_odds.json'
#with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
#    jsonf.write(json.dumps(parsed, indent=4))
#print('saved to '+jsonFilePath)

In [8]:
# add empty date column to prediction_history
prediction_history['date'] = ''
result = prediction_history.to_json()
parsed = json.loads(result)
jsonFilePath='src/content/data/external/prediction_history.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)


saved to src/content/data/external/prediction_history.json


In [9]:
prediction_history

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref,predicted fighter odds,predicted opponent odds,average bookie odds,correct?,date
0,Angela Hill,,145,145,150,144,,+150,150,+150,146,,Mackenzie Dern,,-175,-170,-190,-186,,-190,-175,-188,-171,,305,-305,"[148, -181]",,
1,Anthony Hernandez,,-210,-200,-235,-245,,-235,-225,-250,-191,,Edmen Shahbazyan,,170,170,185,186,,185,188,+200,166,,-139,139,"[-224, 181]",,
2,Emily Ducote,,130,125,128,128,,+128,130,+120,133,,Loopy Godinez,,-160,-150,-159,-164,,-159,-160,-150,-153,,210,-210,"[128, -157]",,
3,Andre Fialho,,185,190,180,172,,+180,188,+175,180,,Joaquin Buckley,,-225,-230,-230,-225,,-230,-225,-225,-210,,157,-157,"[181, -225]",,
4,Diego Ferreira,,-155,-150,-167,-156,,-167,-160,-163,-163,,Michael Johnson,,125,125,135,122,,135,130,+130,143,,-144,144,"[-160, 131]",,
5,Karolina Kowalkiewicz,,-135,-130,-137,-140,,-137,-125,-138,-130,,Vanessa Demopoulos,,110,110,110,110,,110,100,+110,110,,125,-125,"[-134, 109]",,
6,Gilbert Urbina,,100,-105,-105,102,,-105,100,+100,-105,,Orion Cosce,,-120,-115,-118,-130,,-118,-125,-125,-115,,-144,144,"[-2, -121]",,
7,Ilir Latifi,,165,170,165,152,,+165,163,+150,169,,Rodrigo Nascimento,,-200,-200,-210,-196,,-210,-188,-188,-194,,129,-129,"[162, -198]",,
8,Chase Hooper,,110,115,116,108,,+116,110,+110,102,,Nick Fiore,,-135,-135,-143,-138,,-143,-138,-138,-122,,-198,198,"[111, -136]",,
9,Natalia Silva,,-900,-900,-910,-1100,,-910,-800,-800,-1000,,Victoria Leonardo,,575,600,600,600,,600,550,+500,700,,-334,334,"[-915, 591]",,


In [ ]:
prediction_history = pd.concat([vegas_odds, prediction_history], axis = 0).reset_index(drop=True)

In [ ]:
prediction_history[['fighter name','opponent name','predicted fighter odds',	'predicted opponent odds',	'correct?',	'date']]

In [ ]:
# save prediction history to csv
prediction_history.to_csv('src/content/data/processed/prediction_history.csv', index=False)


In [ ]:
# lets clean up prediction history
# first. If two people never fought each other, we can drop the fight
# call this every time BEFORE we merge with vegas_odds
# plug in the prediction_history dataframe and ufc_fights dataframe
    

In [8]:
ufc_fights_crap = pd.read_csv('src/content/data/processed/ufc_fights_crap.csv',low_memory=False)

In [9]:
ufc_fights_crap

,date,fight_url,event_url,result,fighter,opponent,division,method,round,time,fighter_url,opponent_url,knockdowns,sub_attempts,pass,reversals,takedowns_landed,takedowns_attempts,sig_strikes_landed,sig_strikes_attempts,total_strikes_landed,total_strikes_attempts,head_strikes_landed,head_strikes_attempts,body_strikes_landed,body_strikes_attempts,leg_strikes_landed,leg_strikes_attempts,distance_strikes_landed,distance_strikes_attempts,clinch_strikes_landed,clinch_strikes_attempts,ground_strikes_landed,ground_strikes_attempts,control,fighter_wins,fighter_losses,fighter_age,fighter_height,fighter_reach,fighter_L5Y_wins,fighter_L5Y_losses,fighter_L2Y_wins,fighter_L2Y_losses,fighter_ko_wins,fighter_ko_losses,fighter_L5Y_ko_wins,fighter_L5Y_ko_losses,fighter_L2Y_ko_wins,fighter_L2Y_ko_losses,fighter_sub_wins,fighter_sub_losses,fighter_L5Y_sub_wins,fighter_L5Y_sub_losses,fighter_L2Y_sub_wins,fighter_L2Y_sub_losses,fighter_inf_knockdowns_avg,fighter_inf_pass_avg,fighter_inf_reversals_avg,fighter_inf_sub_attempts_avg,fighter_inf_takedowns_landed_avg,fighter_inf_takedowns_attempts_avg,fighter_inf_sig_strikes_landed_avg,fighter_inf_sig_strikes_attempts_avg,fighter_inf_total_strikes_landed_avg,fighter_inf_total_strikes_attempts_avg,fighter_inf_head_strikes_landed_avg,fighter_inf_head_strikes_attempts_avg,fighter_inf_body_strikes_landed_avg,fighter_inf_body_strikes_attempts_avg,fighter_inf_leg_strikes_landed_avg,fighter_inf_leg_strikes_attempts_avg,fighter_inf_distance_strikes_landed_avg,fighter_inf_distance_strikes_attempts_avg,fighter_inf_clinch_strikes_landed_avg,fighter_inf_clinch_strikes_attempts_avg,fighter_inf_ground_strikes_landed_avg,fighter_inf_ground_strikes_attempts_avg,fighter_abs_knockdowns_avg,fighter_abs_pass_avg,fighter_abs_reversals_avg,fighter_abs_sub_attempts_avg,fighter_abs_takedowns_landed_avg,fighter_abs_takedowns_attempts_avg,fighter_abs_sig_strikes_landed_avg,fighter_abs_sig_strikes_attempts_avg,fighter_abs_total_strikes_landed_avg,fighter_abs_total_strikes_attempts_avg,fighter_abs_head_strikes_landed_avg,fighter_abs_head_strikes_attempts_avg,fighter_abs_body_strikes_landed_avg,fighter_abs_body_strikes_attempts_avg,fighter_abs_leg_strikes_landed_avg,fighter_abs_leg_strikes_attempts_avg,fighter_abs_distance_strikes_landed_avg,fighter_abs_distance_strikes_attempts_avg,fighter_abs_clinch_strikes_landed_avg,fighter_abs_clinch_strikes_attempts_avg,fighter_abs_ground_strikes_landed_avg,fighter_abs_ground_strikes_attempts_avg,opponent_wins,opponent_losses,opponent_age,opponent_height,opponent_reach,opponent_L5Y_wins,opponent_L5Y_losses,opponent_L2Y_wins,opponent_L2Y_losses,opponent_ko_wins,opponent_ko_losses,opponent_L5Y_ko_wins,opponent_L5Y_ko_losses,opponent_L2Y_ko_wins,opponent_L2Y_ko_losses,opponent_sub_wins,opponent_sub_losses,opponent_L5Y_sub_wins,opponent_L5Y_sub_losses,opponent_L2Y_sub_wins,opponent_L2Y_sub_losses,opponent_inf_knockdowns_avg,opponent_inf_pass_avg,opponent_inf_reversals_avg,opponent_inf_sub_attempts_avg,opponent_inf_takedowns_landed_avg,opponent_inf_takedowns_attempts_avg,opponent_inf_sig_strikes_landed_avg,opponent_inf_sig_strikes_attempts_avg,opponent_inf_total_strikes_landed_avg,opponent_inf_total_strikes_attempts_avg,opponent_inf_head_strikes_landed_avg,opponent_inf_head_strikes_attempts_avg,opponent_inf_body_strikes_landed_avg,opponent_inf_body_strikes_attempts_avg,opponent_inf_leg_strikes_landed_avg,opponent_inf_leg_strikes_attempts_avg,opponent_inf_distance_strikes_landed_avg,opponent_inf_distance_strikes_attempts_avg,opponent_inf_clinch_strikes_landed_avg,opponent_inf_clinch_strikes_attempts_avg,opponent_inf_ground_strikes_landed_avg,opponent_inf_ground_strikes_attempts_avg,opponent_abs_knockdowns_avg,opponent_abs_pass_avg,opponent_abs_reversals_avg,opponent_abs_sub_attempts_avg,opponent_abs_takedowns_landed_avg,opponent_abs_takedowns_attempts_avg,opponent_abs_sig_strikes_landed_avg,opponent_abs_sig_strikes_attempts_avg,opponent_abs_total_strikes_landed_avg,opponent_abs_total_strikes_attempts_

In [15]:
vegas_odds = pd.read_json('src/content/data/external/vegas_odds.json')

In [27]:
vegas_odds.dtypes

fighter name                       object
fighter DraftKings                 object
fighter BetMGM                     object
fighter Caesars                    object
fighter BetRivers                  object
fighter FanDuel                    object
fighter PointsBet                  object
fighter Unibet                     object
fighter Bet365                     object
fighter BetWay                     object
fighter 5D                         object
fighter Ref                        object
opponent name                      object
opponent DraftKings                object
opponent BetMGM                    object
opponent Caesars                   object
opponent BetRivers                 object
opponent FanDuel                   object
opponent PointsBet                 object
opponent Unibet                    object
opponent Bet365                    object
opponent BetWay                    object
opponent 5D                        object
opponent Ref                      

In [50]:
os.chdir('src')

In [121]:
from functions import *

In [42]:
# find all fights with given date in ufc_fights_crap
# iterate over rows of vegas_odds
i = 0
card_date = vegas_odds['date'][i]
relevant_fights = ufc_fights_crap[pd.to_datetime(ufc_fights_crap['date']) == card_date]

In [52]:
for index1, row1 in vegas_odds.iterrows():
    print(index1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [55]:
prediction_history = pd.read_json('content/data/external/prediction_history.json')

# Testing structure to remove fights from vegas_odds before adding them to prediction history if they didn't actually happen

In [62]:
ufc_fights_crap = pd.read_csv('content/data/processed/ufc_fights_crap.csv',low_memory=False)

#set vegas_odds_old to be the first 20 rows of prediction_history
vegas_odds_old = prediction_history.iloc[:20,:].reset_index(drop=True)

# getting rid of fights that didn't actually happen
bad_indices = []
for index1, row1 in vegas_odds_old.iterrows():
    card_date = row1['date']
    relevant_fights = ufc_fights_crap[pd.to_datetime(ufc_fights_crap['date']) == pd.to_datetime('May 20, 2023')]
    match_found = False
    for index2, row2 in relevant_fights.iterrows():
        if same_name(row1['fighter name'], row2['fighter']) and same_name(row1['opponent name'], row2['opponent']):
            match_found = True
            print('adding fight from '+str(card_date)+' between '+row1['fighter name']+' and '+row1['opponent name'])
            break
    if not match_found:
        bad_indices.append(index1)
        print('fight from '+str(card_date)+' between '+row1['fighter name']+' and '+row1['opponent name'] + ' not found in ufc_fights_crap.csv')
# drop bad indices
vegas_odds_old = vegas_odds_old.drop(bad_indices)
    

adding fight from NaT between Angela Hill and Mackenzie Dern
adding fight from NaT between Anthony Hernandez and Edmen Shahbazyan
adding fight from NaT between Emily Ducote and Loopy Godinez
adding fight from NaT between Andre Fialho and Joaquin Buckley
adding fight from NaT between Diego Ferreira and Michael Johnson
adding fight from NaT between Karolina Kowalkiewicz and Vanessa Demopoulos
adding fight from NaT between Gilbert Urbina and Orion Cosce
adding fight from NaT between Ilir Latifi and Rodrigo Nascimento
adding fight from NaT between Chase Hooper and Nick Fiore
adding fight from NaT between Natalia Silva and Victoria Leonardo
adding fight from NaT between Takashi Sato and Themba Gorimbo
fight from NaT between Amir Albazi and Kai Kara-France not found in ufc_fights_crap.csv
fight from NaT between Alex Caceres and Daniel Pineda not found in ufc_fights_crap.csv
fight from NaT between Jim Miller and Ludovit Klein not found in ufc_fights_crap.csv
fight from NaT between Guram Kutat

In [60]:
ufc_fights_crap

,date,fight_url,event_url,result,fighter,opponent,division,method,round,time,fighter_url,opponent_url,knockdowns,sub_attempts,pass,reversals,takedowns_landed,takedowns_attempts,sig_strikes_landed,sig_strikes_attempts,total_strikes_landed,total_strikes_attempts,head_strikes_landed,head_strikes_attempts,body_strikes_landed,body_strikes_attempts,leg_strikes_landed,leg_strikes_attempts,distance_strikes_landed,distance_strikes_attempts,clinch_strikes_landed,clinch_strikes_attempts,ground_strikes_landed,ground_strikes_attempts,control,fighter_wins,fighter_losses,fighter_age,fighter_height,fighter_reach,fighter_L5Y_wins,fighter_L5Y_losses,fighter_L2Y_wins,fighter_L2Y_losses,fighter_ko_wins,fighter_ko_losses,fighter_L5Y_ko_wins,fighter_L5Y_ko_losses,fighter_L2Y_ko_wins,fighter_L2Y_ko_losses,fighter_sub_wins,fighter_sub_losses,fighter_L5Y_sub_wins,fighter_L5Y_sub_losses,fighter_L2Y_sub_wins,fighter_L2Y_sub_losses,fighter_inf_knockdowns_avg,fighter_inf_pass_avg,fighter_inf_reversals_avg,fighter_inf_sub_attempts_avg,fighter_inf_takedowns_landed_avg,fighter_inf_takedowns_attempts_avg,fighter_inf_sig_strikes_landed_avg,fighter_inf_sig_strikes_attempts_avg,fighter_inf_total_strikes_landed_avg,fighter_inf_total_strikes_attempts_avg,fighter_inf_head_strikes_landed_avg,fighter_inf_head_strikes_attempts_avg,fighter_inf_body_strikes_landed_avg,fighter_inf_body_strikes_attempts_avg,fighter_inf_leg_strikes_landed_avg,fighter_inf_leg_strikes_attempts_avg,fighter_inf_distance_strikes_landed_avg,fighter_inf_distance_strikes_attempts_avg,fighter_inf_clinch_strikes_landed_avg,fighter_inf_clinch_strikes_attempts_avg,fighter_inf_ground_strikes_landed_avg,fighter_inf_ground_strikes_attempts_avg,fighter_abs_knockdowns_avg,fighter_abs_pass_avg,fighter_abs_reversals_avg,fighter_abs_sub_attempts_avg,fighter_abs_takedowns_landed_avg,fighter_abs_takedowns_attempts_avg,fighter_abs_sig_strikes_landed_avg,fighter_abs_sig_strikes_attempts_avg,fighter_abs_total_strikes_landed_avg,fighter_abs_total_strikes_attempts_avg,fighter_abs_head_strikes_landed_avg,fighter_abs_head_strikes_attempts_avg,fighter_abs_body_strikes_landed_avg,fighter_abs_body_strikes_attempts_avg,fighter_abs_leg_strikes_landed_avg,fighter_abs_leg_strikes_attempts_avg,fighter_abs_distance_strikes_landed_avg,fighter_abs_distance_strikes_attempts_avg,fighter_abs_clinch_strikes_landed_avg,fighter_abs_clinch_strikes_attempts_avg,fighter_abs_ground_strikes_landed_avg,fighter_abs_ground_strikes_attempts_avg,opponent_wins,opponent_losses,opponent_age,opponent_height,opponent_reach,opponent_L5Y_wins,opponent_L5Y_losses,opponent_L2Y_wins,opponent_L2Y_losses,opponent_ko_wins,opponent_ko_losses,opponent_L5Y_ko_wins,opponent_L5Y_ko_losses,opponent_L2Y_ko_wins,opponent_L2Y_ko_losses,opponent_sub_wins,opponent_sub_losses,opponent_L5Y_sub_wins,opponent_L5Y_sub_losses,opponent_L2Y_sub_wins,opponent_L2Y_sub_losses,opponent_inf_knockdowns_avg,opponent_inf_pass_avg,opponent_inf_reversals_avg,opponent_inf_sub_attempts_avg,opponent_inf_takedowns_landed_avg,opponent_inf_takedowns_attempts_avg,opponent_inf_sig_strikes_landed_avg,opponent_inf_sig_strikes_attempts_avg,opponent_inf_total_strikes_landed_avg,opponent_inf_total_strikes_attempts_avg,opponent_inf_head_strikes_landed_avg,opponent_inf_head_strikes_attempts_avg,opponent_inf_body_strikes_landed_avg,opponent_inf_body_strikes_attempts_avg,opponent_inf_leg_strikes_landed_avg,opponent_inf_leg_strikes_attempts_avg,opponent_inf_distance_strikes_landed_avg,opponent_inf_distance_strikes_attempts_avg,opponent_inf_clinch_strikes_landed_avg,opponent_inf_clinch_strikes_attempts_avg,opponent_inf_ground_strikes_landed_avg,opponent_inf_ground_strikes_attempts_avg,opponent_abs_knockdowns_avg,opponent_abs_pass_avg,opponent_abs_reversals_avg,opponent_abs_sub_attempts_avg,opponent_abs_takedowns_landed_avg,opponent_abs_takedowns_attempts_avg,opponent_abs_sig_strikes_landed_avg,opponent_abs_sig_strikes_attempts_avg,opponent_abs_total_strikes_landed_avg,opponent_abs_total_strikes_attempts_

# We use the same type of function to clean prediction_history

# Can run this next series of cells if prediction_history somehow becomes full of fights that didn't happen.

But hopefully the new structure in update_and_rebuild_model will fix this

In [79]:
ufc_fights_crap = pd.read_csv('content/data/processed/ufc_fights_crap.csv',low_memory=False)

#set vegas_odds_old to be the first 20 rows of prediction_history

# getting rid of fights that didn't actually happen
bad_indices = []
for index1, row1 in prediction_history.iterrows():
    relevant_fights = ufc_fights_crap
    match_found = False
    fighter_odds = row1['predicted fighter odds']
    for index2, row2 in relevant_fights.iterrows():
        if same_name(row1['fighter name'], row2['fighter']) and same_name(row1['opponent name'], row2['opponent']):
            match_found = True
            print('adding fight from '+str(card_date)+' between '+row1['fighter name']+' and '+row1['opponent name'])
            if (fighter_odds < 0 and row2['result'] == 'W') or (fighter_odds > 0 and row2['result'] == 'L'):
                prediction_history.at[index1,'correct?'] = 1
            else:
                prediction_history.at[index1,'correct?'] = 0
            break
    if not match_found:
        bad_indices.append(index1)
        print('fight from '+str(card_date)+' between '+row1['fighter name']+' and '+row1['opponent name'] + ' not found in ufc_fights_crap.csv')

adding fight from NaT between Angela Hill and Mackenzie Dern
adding fight from NaT between Anthony Hernandez and Edmen Shahbazyan
adding fight from NaT between Emily Ducote and Loopy Godinez
adding fight from NaT between Andre Fialho and Joaquin Buckley
adding fight from NaT between Diego Ferreira and Michael Johnson
adding fight from NaT between Karolina Kowalkiewicz and Vanessa Demopoulos
adding fight from NaT between Gilbert Urbina and Orion Cosce
adding fight from NaT between Ilir Latifi and Rodrigo Nascimento
adding fight from NaT between Chase Hooper and Nick Fiore
adding fight from NaT between Natalia Silva and Victoria Leonardo
adding fight from NaT between Takashi Sato and Themba Gorimbo
adding fight from NaT between Amir Albazi and Kai Kara-France
adding fight from NaT between Alex Caceres and Daniel Pineda
adding fight from NaT between Andrei Arlovski and Don'tale Mayes
adding fight from NaT between Daniel Santos and Johnny Munoz
adding fight from NaT between Elise Reed and 

In [82]:
bad_indices

[]

In [66]:
prediction_history = prediction_history.drop(bad_indices)

In [83]:
result = prediction_history.to_json()
parsed = json.loads(result)
jsonFilePath='content/data/external/prediction_history.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)

saved to content/data/external/prediction_history.json


In [46]:
ufc_fights_crap[pd.to_datetime(ufc_fights_crap['date']) == pd.to_datetime('July 08, 2023')]

,date,fight_url,event_url,result,fighter,opponent,division,method,round,time,fighter_url,opponent_url,knockdowns,sub_attempts,pass,reversals,takedowns_landed,takedowns_attempts,sig_strikes_landed,sig_strikes_attempts,total_strikes_landed,total_strikes_attempts,head_strikes_landed,head_strikes_attempts,body_strikes_landed,body_strikes_attempts,leg_strikes_landed,leg_strikes_attempts,distance_strikes_landed,distance_strikes_attempts,clinch_strikes_landed,clinch_strikes_attempts,ground_strikes_landed,ground_strikes_attempts,control,fighter_wins,fighter_losses,fighter_age,fighter_height,fighter_reach,fighter_L5Y_wins,fighter_L5Y_losses,fighter_L2Y_wins,fighter_L2Y_losses,fighter_ko_wins,fighter_ko_losses,fighter_L5Y_ko_wins,fighter_L5Y_ko_losses,fighter_L2Y_ko_wins,fighter_L2Y_ko_losses,fighter_sub_wins,fighter_sub_losses,fighter_L5Y_sub_wins,fighter_L5Y_sub_losses,fighter_L2Y_sub_wins,fighter_L2Y_sub_losses,fighter_inf_knockdowns_avg,fighter_inf_pass_avg,fighter_inf_reversals_avg,fighter_inf_sub_attempts_avg,fighter_inf_takedowns_landed_avg,fighter_inf_takedowns_attempts_avg,fighter_inf_sig_strikes_landed_avg,fighter_inf_sig_strikes_attempts_avg,fighter_inf_total_strikes_landed_avg,fighter_inf_total_strikes_attempts_avg,fighter_inf_head_strikes_landed_avg,fighter_inf_head_strikes_attempts_avg,fighter_inf_body_strikes_landed_avg,fighter_inf_body_strikes_attempts_avg,fighter_inf_leg_strikes_landed_avg,fighter_inf_leg_strikes_attempts_avg,fighter_inf_distance_strikes_landed_avg,fighter_inf_distance_strikes_attempts_avg,fighter_inf_clinch_strikes_landed_avg,fighter_inf_clinch_strikes_attempts_avg,fighter_inf_ground_strikes_landed_avg,fighter_inf_ground_strikes_attempts_avg,fighter_abs_knockdowns_avg,fighter_abs_pass_avg,fighter_abs_reversals_avg,fighter_abs_sub_attempts_avg,fighter_abs_takedowns_landed_avg,fighter_abs_takedowns_attempts_avg,fighter_abs_sig_strikes_landed_avg,fighter_abs_sig_strikes_attempts_avg,fighter_abs_total_strikes_landed_avg,fighter_abs_total_strikes_attempts_avg,fighter_abs_head_strikes_landed_avg,fighter_abs_head_strikes_attempts_avg,fighter_abs_body_strikes_landed_avg,fighter_abs_body_strikes_attempts_avg,fighter_abs_leg_strikes_landed_avg,fighter_abs_leg_strikes_attempts_avg,fighter_abs_distance_strikes_landed_avg,fighter_abs_distance_strikes_attempts_avg,fighter_abs_clinch_strikes_landed_avg,fighter_abs_clinch_strikes_attempts_avg,fighter_abs_ground_strikes_landed_avg,fighter_abs_ground_strikes_attempts_avg,opponent_wins,opponent_losses,opponent_age,opponent_height,opponent_reach,opponent_L5Y_wins,opponent_L5Y_losses,opponent_L2Y_wins,opponent_L2Y_losses,opponent_ko_wins,opponent_ko_losses,opponent_L5Y_ko_wins,opponent_L5Y_ko_losses,opponent_L2Y_ko_wins,opponent_L2Y_ko_losses,opponent_sub_wins,opponent_sub_losses,opponent_L5Y_sub_wins,opponent_L5Y_sub_losses,opponent_L2Y_sub_wins,opponent_L2Y_sub_losses,opponent_inf_knockdowns_avg,opponent_inf_pass_avg,opponent_inf_reversals_avg,opponent_inf_sub_attempts_avg,opponent_inf_takedowns_landed_avg,opponent_inf_takedowns_attempts_avg,opponent_inf_sig_strikes_landed_avg,opponent_inf_sig_strikes_attempts_avg,opponent_inf_total_strikes_landed_avg,opponent_inf_total_strikes_attempts_avg,opponent_inf_head_strikes_landed_avg,opponent_inf_head_strikes_attempts_avg,opponent_inf_body_strikes_landed_avg,opponent_inf_body_strikes_attempts_avg,opponent_inf_leg_strikes_landed_avg,opponent_inf_leg_strikes_attempts_avg,opponent_inf_distance_strikes_landed_avg,opponent_inf_distance_strikes_attempts_avg,opponent_inf_clinch_strikes_landed_avg,opponent_inf_clinch_strikes_attempts_avg,opponent_inf_ground_strikes_landed_avg,opponent_inf_ground_strikes_attempts_avg,opponent_abs_knockdowns_avg,opponent_abs_pass_avg,opponent_abs_reversals_avg,opponent_abs_sub_attempts_avg,opponent_abs_takedowns_landed_avg,opponent_abs_takedowns_attempts_avg,opponent_abs_sig_strikes_landed_avg,opponent_abs_sig_strikes_attempts_avg,opponent_abs_total_strikes_landed_avg,opponent_abs_total_strikes_attempts_

In [38]:
ufc_fights_crap['date'][0]
# convert to Timestamp

'July 08, 2023'

In [39]:
fight_date = pd.to_datetime('July 15, 2023')

In [40]:
card_date

Timestamp('2023-07-15 00:00:00')

In [41]:
fight_date == card_date

True

In [17]:
relevant_fights

,date,fight_url,event_url,result,fighter,opponent,division,method,round,time,fighter_url,opponent_url,knockdowns,sub_attempts,pass,reversals,takedowns_landed,takedowns_attempts,sig_strikes_landed,sig_strikes_attempts,total_strikes_landed,total_strikes_attempts,head_strikes_landed,head_strikes_attempts,body_strikes_landed,body_strikes_attempts,leg_strikes_landed,leg_strikes_attempts,distance_strikes_landed,distance_strikes_attempts,clinch_strikes_landed,clinch_strikes_attempts,ground_strikes_landed,ground_strikes_attempts,control,fighter_wins,fighter_losses,fighter_age,fighter_height,fighter_reach,fighter_L5Y_wins,fighter_L5Y_losses,fighter_L2Y_wins,fighter_L2Y_losses,fighter_ko_wins,fighter_ko_losses,fighter_L5Y_ko_wins,fighter_L5Y_ko_losses,fighter_L2Y_ko_wins,fighter_L2Y_ko_losses,fighter_sub_wins,fighter_sub_losses,fighter_L5Y_sub_wins,fighter_L5Y_sub_losses,fighter_L2Y_sub_wins,fighter_L2Y_sub_losses,fighter_inf_knockdowns_avg,fighter_inf_pass_avg,fighter_inf_reversals_avg,fighter_inf_sub_attempts_avg,fighter_inf_takedowns_landed_avg,fighter_inf_takedowns_attempts_avg,fighter_inf_sig_strikes_landed_avg,fighter_inf_sig_strikes_attempts_avg,fighter_inf_total_strikes_landed_avg,fighter_inf_total_strikes_attempts_avg,fighter_inf_head_strikes_landed_avg,fighter_inf_head_strikes_attempts_avg,fighter_inf_body_strikes_landed_avg,fighter_inf_body_strikes_attempts_avg,fighter_inf_leg_strikes_landed_avg,fighter_inf_leg_strikes_attempts_avg,fighter_inf_distance_strikes_landed_avg,fighter_inf_distance_strikes_attempts_avg,fighter_inf_clinch_strikes_landed_avg,fighter_inf_clinch_strikes_attempts_avg,fighter_inf_ground_strikes_landed_avg,fighter_inf_ground_strikes_attempts_avg,fighter_abs_knockdowns_avg,fighter_abs_pass_avg,fighter_abs_reversals_avg,fighter_abs_sub_attempts_avg,fighter_abs_takedowns_landed_avg,fighter_abs_takedowns_attempts_avg,fighter_abs_sig_strikes_landed_avg,fighter_abs_sig_strikes_attempts_avg,fighter_abs_total_strikes_landed_avg,fighter_abs_total_strikes_attempts_avg,fighter_abs_head_strikes_landed_avg,fighter_abs_head_strikes_attempts_avg,fighter_abs_body_strikes_landed_avg,fighter_abs_body_strikes_attempts_avg,fighter_abs_leg_strikes_landed_avg,fighter_abs_leg_strikes_attempts_avg,fighter_abs_distance_strikes_landed_avg,fighter_abs_distance_strikes_attempts_avg,fighter_abs_clinch_strikes_landed_avg,fighter_abs_clinch_strikes_attempts_avg,fighter_abs_ground_strikes_landed_avg,fighter_abs_ground_strikes_attempts_avg,opponent_wins,opponent_losses,opponent_age,opponent_height,opponent_reach,opponent_L5Y_wins,opponent_L5Y_losses,opponent_L2Y_wins,opponent_L2Y_losses,opponent_ko_wins,opponent_ko_losses,opponent_L5Y_ko_wins,opponent_L5Y_ko_losses,opponent_L2Y_ko_wins,opponent_L2Y_ko_losses,opponent_sub_wins,opponent_sub_losses,opponent_L5Y_sub_wins,opponent_L5Y_sub_losses,opponent_L2Y_sub_wins,opponent_L2Y_sub_losses,opponent_inf_knockdowns_avg,opponent_inf_pass_avg,opponent_inf_reversals_avg,opponent_inf_sub_attempts_avg,opponent_inf_takedowns_landed_avg,opponent_inf_takedowns_attempts_avg,opponent_inf_sig_strikes_landed_avg,opponent_inf_sig_strikes_attempts_avg,opponent_inf_total_strikes_landed_avg,opponent_inf_total_strikes_attempts_avg,opponent_inf_head_strikes_landed_avg,opponent_inf_head_strikes_attempts_avg,opponent_inf_body_strikes_landed_avg,opponent_inf_body_strikes_attempts_avg,opponent_inf_leg_strikes_landed_avg,opponent_inf_leg_strikes_attempts_avg,opponent_inf_distance_strikes_landed_avg,opponent_inf_distance_strikes_attempts_avg,opponent_inf_clinch_strikes_landed_avg,opponent_inf_clinch_strikes_attempts_avg,opponent_inf_ground_strikes_landed_avg,opponent_inf_ground_strikes_attempts_avg,opponent_abs_knockdowns_avg,opponent_abs_pass_avg,opponent_abs_reversals_avg,opponent_abs_sub_attempts_avg,opponent_abs_takedowns_landed_avg,opponent_abs_takedowns_attempts_avg,opponent_abs_sig_strikes_landed_avg,opponent_abs_sig_strikes_attempts_avg,opponent_abs_total_strikes_landed_avg,opponent_abs_total_strikes_attempts_

In [18]:
card_date

Timestamp('2023-07-15 00:00:00')

In [21]:
example_date = ufc_fights_crap['date'][0]

In [ ]:
#convert example date to datetime object


# Make predictions for fights we missed

In [94]:
new_fights = ufc_fights_crap.loc[:147]

In [95]:
import random
random_indices = []
for i in range(0, len(new_fights['fighter']), 2):
    random_indices.append(random.choice([i, i+1]))

new_fights = new_fights.drop(random_indices)

In [96]:
new_fights.shape[0]

74

In [97]:
new_fights

,date,fight_url,event_url,result,fighter,opponent,division,method,round,time,fighter_url,opponent_url,knockdowns,sub_attempts,pass,reversals,takedowns_landed,takedowns_attempts,sig_strikes_landed,sig_strikes_attempts,total_strikes_landed,total_strikes_attempts,head_strikes_landed,head_strikes_attempts,body_strikes_landed,body_strikes_attempts,leg_strikes_landed,leg_strikes_attempts,distance_strikes_landed,distance_strikes_attempts,clinch_strikes_landed,clinch_strikes_attempts,ground_strikes_landed,ground_strikes_attempts,control,fighter_wins,fighter_losses,fighter_age,fighter_height,fighter_reach,fighter_L5Y_wins,fighter_L5Y_losses,fighter_L2Y_wins,fighter_L2Y_losses,fighter_ko_wins,fighter_ko_losses,fighter_L5Y_ko_wins,fighter_L5Y_ko_losses,fighter_L2Y_ko_wins,fighter_L2Y_ko_losses,fighter_sub_wins,fighter_sub_losses,fighter_L5Y_sub_wins,fighter_L5Y_sub_losses,fighter_L2Y_sub_wins,fighter_L2Y_sub_losses,fighter_inf_knockdowns_avg,fighter_inf_pass_avg,fighter_inf_reversals_avg,fighter_inf_sub_attempts_avg,fighter_inf_takedowns_landed_avg,fighter_inf_takedowns_attempts_avg,fighter_inf_sig_strikes_landed_avg,fighter_inf_sig_strikes_attempts_avg,fighter_inf_total_strikes_landed_avg,fighter_inf_total_strikes_attempts_avg,fighter_inf_head_strikes_landed_avg,fighter_inf_head_strikes_attempts_avg,fighter_inf_body_strikes_landed_avg,fighter_inf_body_strikes_attempts_avg,fighter_inf_leg_strikes_landed_avg,fighter_inf_leg_strikes_attempts_avg,fighter_inf_distance_strikes_landed_avg,fighter_inf_distance_strikes_attempts_avg,fighter_inf_clinch_strikes_landed_avg,fighter_inf_clinch_strikes_attempts_avg,fighter_inf_ground_strikes_landed_avg,fighter_inf_ground_strikes_attempts_avg,fighter_abs_knockdowns_avg,fighter_abs_pass_avg,fighter_abs_reversals_avg,fighter_abs_sub_attempts_avg,fighter_abs_takedowns_landed_avg,fighter_abs_takedowns_attempts_avg,fighter_abs_sig_strikes_landed_avg,fighter_abs_sig_strikes_attempts_avg,fighter_abs_total_strikes_landed_avg,fighter_abs_total_strikes_attempts_avg,fighter_abs_head_strikes_landed_avg,fighter_abs_head_strikes_attempts_avg,fighter_abs_body_strikes_landed_avg,fighter_abs_body_strikes_attempts_avg,fighter_abs_leg_strikes_landed_avg,fighter_abs_leg_strikes_attempts_avg,fighter_abs_distance_strikes_landed_avg,fighter_abs_distance_strikes_attempts_avg,fighter_abs_clinch_strikes_landed_avg,fighter_abs_clinch_strikes_attempts_avg,fighter_abs_ground_strikes_landed_avg,fighter_abs_ground_strikes_attempts_avg,opponent_wins,opponent_losses,opponent_age,opponent_height,opponent_reach,opponent_L5Y_wins,opponent_L5Y_losses,opponent_L2Y_wins,opponent_L2Y_losses,opponent_ko_wins,opponent_ko_losses,opponent_L5Y_ko_wins,opponent_L5Y_ko_losses,opponent_L2Y_ko_wins,opponent_L2Y_ko_losses,opponent_sub_wins,opponent_sub_losses,opponent_L5Y_sub_wins,opponent_L5Y_sub_losses,opponent_L2Y_sub_wins,opponent_L2Y_sub_losses,opponent_inf_knockdowns_avg,opponent_inf_pass_avg,opponent_inf_reversals_avg,opponent_inf_sub_attempts_avg,opponent_inf_takedowns_landed_avg,opponent_inf_takedowns_attempts_avg,opponent_inf_sig_strikes_landed_avg,opponent_inf_sig_strikes_attempts_avg,opponent_inf_total_strikes_landed_avg,opponent_inf_total_strikes_attempts_avg,opponent_inf_head_strikes_landed_avg,opponent_inf_head_strikes_attempts_avg,opponent_inf_body_strikes_landed_avg,opponent_inf_body_strikes_attempts_avg,opponent_inf_leg_strikes_landed_avg,opponent_inf_leg_strikes_attempts_avg,opponent_inf_distance_strikes_landed_avg,opponent_inf_distance_strikes_attempts_avg,opponent_inf_clinch_strikes_landed_avg,opponent_inf_clinch_strikes_attempts_avg,opponent_inf_ground_strikes_landed_avg,opponent_inf_ground_strikes_attempts_avg,opponent_abs_knockdowns_avg,opponent_abs_pass_avg,opponent_abs_reversals_avg,opponent_abs_sub_attempts_avg,opponent_abs_takedowns_landed_avg,opponent_abs_takedowns_attempts_avg,opponent_abs_sig_strikes_landed_avg,opponent_abs_sig_strikes_attempts_avg,opponent_abs_total_strikes_landed_avg,opponent_abs_total_strikes_attempts_

In [101]:
#getting dependencies
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import random, sklearn, scipy, json, itertools
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
#this imports all of the functions from the file functions.py
from functions import *

pd.options.mode.chained_assignment = None  # default='warn' (disables SettingWithCopyWarning)

print('importing dataframe from ufc_fights.csv')
#importing csv fight data and saving as dataframes
ufc_fights_winner = pd.read_csv('content/data/processed/ufc_fights.csv',low_memory=False)
ufc_fights_method = pd.read_csv('content/data/processed/ufc_fights.csv',low_memory=False)
ufcfighterscrap =pd.read_csv('content/data/processed/fighter_stats.csv',sep=',',low_memory=False)

#there are some issues with how names are saved
#it gets saved as Joanne Wood for some reason
for i in range(len(ufc_fights_winner['fighter'])):
    if ufc_fights_winner['fighter'][i]=='Joanne Wood':
        ufc_fights_winner['fighter'][i]='Joanne Calderwood'
    if ufc_fights_winner['opponent'][i]=='Joanne Wood':
        ufc_fights_winner['opponent'][i]='Joanne Calderwood'

#there are two lists of names which sometimes get scraped differently... this causes some problems which are being
#addressed in this cell.

#cleaning the methods column for winner prediction
#changing anything other than 'U-DEC','M-DEC', 'KO/TKO', 'SUB', to 'bullshit'
#changing 'U-DEC','M-DEC', to 'DEC'
#(counting split decisions as bullshit)
ufc_fights_winner['method'] = clean_method_for_winner_vect(ufc_fights_winner['method'])
ufc_fights_winner['method'].unique()

#cleaning the methods column for method prediction
#changing anything other than 'U-DEC','M-DEC', 'S-DEC', 'KO/TKO', 'SUB', to 'bullshit'
#changing 'U-DEC','M-DEC', 'S-DEC', to 'DEC'
#(counting split decisions as decisions)
ufc_fights_method['method'] = clean_method_vect(ufc_fights_method['method'])
ufc_fights_method['method'].unique()

#getting rid of rows with incomplete or useless data
#fights with outcome "Win" or "Loss" (no "Draw")
draw_mask=ufc_fights_winner['result'] != 'D'
#fights where the method of victory is TKO/SUB/DEC (no split decision or DQ or Overturned or anything else like that)
method_mask_winner=(ufc_fights_winner['method']!='bullshit')
method_mask_method=(ufc_fights_method['method']!='bullshit')
#fights where age is known
age_mask=(ufc_fights_winner['fighter_age']!='unknown')&(ufc_fights_winner['opponent_age']!='unknown')&(ufc_fights_winner['fighter_age']!=0)&(ufc_fights_winner['opponent_age']!=0)
#fights where height reach is known
height_mask=(ufc_fights_winner['fighter_height']!='unknown')&(ufc_fights_winner['opponent_height']!='unknown')
reach_mask=(ufc_fights_winner['fighter_reach']!='unknown')&(ufc_fights_winner['opponent_reach']!='unknown')
#fights where number of wins is known
wins_mask=(ufc_fights_winner['fighter_wins'] != 'unknown' )& (ufc_fights_winner['opponent_wins'] != 'unknown')
#fights where both fighters have strike statistics (gets rid of UFC debuts)
strikes_mask=(ufc_fights_winner['fighter_inf_sig_strikes_attempts_avg'] != 0)&(ufc_fights_winner['opponent_inf_sig_strikes_attempts_avg'] != 0)
#includes only the fights satisfying these conditions
ufc_fights_winner=ufc_fights_winner[draw_mask&method_mask_winner&age_mask&height_mask&reach_mask&wins_mask&strikes_mask]
ufc_fights_method=ufc_fights_method[draw_mask&method_mask_method&age_mask&height_mask&reach_mask&wins_mask&strikes_mask]
ufc_fights=ufc_fights[draw_mask&method_mask_winner&age_mask&height_mask&reach_mask&wins_mask&strikes_mask]

#listing all stats and making some new stats from them (differences often score higher in the learning models)
record_statistics=[u'fighter_wins',u'fighter_losses',u'fighter_L5Y_wins',u'fighter_L5Y_losses',u'fighter_L2Y_wins',u'fighter_L2Y_losses',u'fighter_ko_wins',u'fighter_ko_losses',u'fighter_L5Y_ko_wins',u'fighter_L5Y_ko_losses',u'fighter_L2Y_ko_wins',u'fighter_L2Y_ko_losses',u'fighter_sub_wins',u'fighter_sub_losses',u'fighter_L5Y_sub_wins',u'fighter_L5Y_sub_losses',u'fighter_L2Y_sub_wins',u'fighter_L2Y_sub_losses',u'opponent_wins',u'opponent_losses',u'opponent_L5Y_wins',u'opponent_L5Y_losses',u'opponent_L2Y_wins',u'opponent_L2Y_losses',u'opponent_ko_wins',u'opponent_ko_losses',u'opponent_L5Y_ko_wins',u'opponent_L5Y_ko_losses',u'opponent_L2Y_ko_wins',u'opponent_L2Y_ko_losses',u'opponent_sub_wins',u'opponent_sub_losses',u'opponent_L5Y_sub_wins',u'opponent_L5Y_sub_losses',u'opponent_L2Y_sub_wins',u'opponent_L2Y_sub_losses']

physical_stats=[ u'fighter_age',u'fighter_height',u'fighter_reach',u'opponent_age',u'opponent_height',u'opponent_reach']

#THERE MAY BE A PROBLEM IN AGE HEIGHT REACH TO DO WITH STRING VS FLOAT. MAKE SURE THESE ARE ALL THE CORRECT TYPE
#MAYBE WE ARE LOSING PREDICTABILITY HERE

#here is the list of all stats available (besides stance), does not include names or result
punch_statistics=[  u'fighter_inf_knockdowns_avg',u'fighter_inf_pass_avg',u'fighter_inf_reversals_avg',u'fighter_inf_sub_attempts_avg',u'fighter_inf_takedowns_landed_avg',u'fighter_inf_takedowns_attempts_avg',u'fighter_inf_sig_strikes_landed_avg',u'fighter_inf_sig_strikes_attempts_avg',u'fighter_inf_total_strikes_landed_avg',u'fighter_inf_total_strikes_attempts_avg',u'fighter_inf_head_strikes_landed_avg',u'fighter_inf_head_strikes_attempts_avg',u'fighter_inf_body_strikes_landed_avg',u'fighter_inf_body_strikes_attempts_avg',u'fighter_inf_leg_strikes_landed_avg',u'fighter_inf_leg_strikes_attempts_avg',u'fighter_inf_distance_strikes_landed_avg',u'fighter_inf_distance_strikes_attempts_avg',u'fighter_inf_clinch_strikes_landed_avg',u'fighter_inf_clinch_strikes_attempts_avg',u'fighter_inf_ground_strikes_landed_avg',u'fighter_inf_ground_strikes_attempts_avg',u'fighter_abs_knockdowns_avg',u'fighter_abs_pass_avg',u'fighter_abs_reversals_avg',u'fighter_abs_sub_attempts_avg',u'fighter_abs_takedowns_landed_avg',u'fighter_abs_takedowns_attempts_avg',u'fighter_abs_sig_strikes_landed_avg',u'fighter_abs_sig_strikes_attempts_avg',u'fighter_abs_total_strikes_landed_avg',u'fighter_abs_total_strikes_attempts_avg',u'fighter_abs_head_strikes_landed_avg',u'fighter_abs_head_strikes_attempts_avg',u'fighter_abs_body_strikes_landed_avg',u'fighter_abs_body_strikes_attempts_avg',u'fighter_abs_leg_strikes_landed_avg',u'fighter_abs_leg_strikes_attempts_avg',u'fighter_abs_distance_strikes_landed_avg',u'fighter_abs_distance_strikes_attempts_avg',u'fighter_abs_clinch_strikes_landed_avg',u'fighter_abs_clinch_strikes_attempts_avg',u'fighter_abs_ground_strikes_landed_avg',u'fighter_abs_ground_strikes_attempts_avg',u'opponent_inf_knockdowns_avg',u'opponent_inf_pass_avg',u'opponent_inf_reversals_avg',u'opponent_inf_sub_attempts_avg',u'opponent_inf_takedowns_landed_avg',u'opponent_inf_takedowns_attempts_avg',u'opponent_inf_sig_strikes_landed_avg',u'opponent_inf_sig_strikes_attempts_avg',u'opponent_inf_total_strikes_landed_avg',u'opponent_inf_total_strikes_attempts_avg',u'opponent_inf_head_strikes_landed_avg',u'opponent_inf_head_strikes_attempts_avg',u'opponent_inf_body_strikes_landed_avg',u'opponent_inf_body_strikes_attempts_avg',u'opponent_inf_leg_strikes_landed_avg',u'opponent_inf_leg_strikes_attempts_avg',u'opponent_inf_distance_strikes_landed_avg',u'opponent_inf_distance_strikes_attempts_avg',u'opponent_inf_clinch_strikes_landed_avg',u'opponent_inf_clinch_strikes_attempts_avg',u'opponent_inf_ground_strikes_landed_avg',u'opponent_inf_ground_strikes_attempts_avg',u'opponent_abs_knockdowns_avg',u'opponent_abs_pass_avg',u'opponent_abs_reversals_avg',u'opponent_abs_sub_attempts_avg',u'opponent_abs_takedowns_landed_avg',u'opponent_abs_takedowns_attempts_avg',u'opponent_abs_sig_strikes_landed_avg',u'opponent_abs_sig_strikes_attempts_avg',u'opponent_abs_total_strikes_landed_avg',u'opponent_abs_total_strikes_attempts_avg',u'opponent_abs_head_strikes_landed_avg',u'opponent_abs_head_strikes_attempts_avg',u'opponent_abs_body_strikes_landed_avg',u'opponent_abs_body_strikes_attempts_avg',u'opponent_abs_leg_strikes_landed_avg',u'opponent_abs_leg_strikes_attempts_avg',u'opponent_abs_distance_strikes_landed_avg',u'opponent_abs_distance_strikes_attempts_avg',u'opponent_abs_clinch_strikes_landed_avg',u'opponent_abs_clinch_strikes_attempts_avg',u'opponent_abs_ground_strikes_landed_avg',u'opponent_abs_ground_strikes_attempts_avg']

#here is the version of punch stats geared for comparing fighter_inf to opponent_abs
punch_statistics_alt=[    u'fighter_inf_knockdowns_avg',u'fighter_inf_pass_avg',u'fighter_inf_reversals_avg',u'fighter_inf_sub_attempts_avg',u'fighter_inf_takedowns_landed_avg',u'fighter_inf_takedowns_attempts_avg',u'fighter_inf_sig_strikes_landed_avg',u'fighter_inf_sig_strikes_attempts_avg',u'fighter_inf_total_strikes_landed_avg',u'fighter_inf_total_strikes_attempts_avg',u'fighter_inf_head_strikes_landed_avg',u'fighter_inf_head_strikes_attempts_avg',u'fighter_inf_body_strikes_landed_avg',u'fighter_inf_body_strikes_attempts_avg',u'fighter_inf_leg_strikes_landed_avg',u'fighter_inf_leg_strikes_attempts_avg',u'fighter_inf_distance_strikes_landed_avg',u'fighter_inf_distance_strikes_attempts_avg',u'fighter_inf_clinch_strikes_landed_avg',u'fighter_inf_clinch_strikes_attempts_avg',u'fighter_inf_ground_strikes_landed_avg',u'fighter_inf_ground_strikes_attempts_avg',
u'fighter_abs_knockdowns_avg',u'fighter_abs_pass_avg',u'fighter_abs_reversals_avg',u'fighter_abs_sub_attempts_avg',u'fighter_abs_takedowns_landed_avg',u'fighter_abs_takedowns_attempts_avg',u'fighter_abs_sig_strikes_landed_avg',u'fighter_abs_sig_strikes_attempts_avg',u'fighter_abs_total_strikes_landed_avg',u'fighter_abs_total_strikes_attempts_avg',u'fighter_abs_head_strikes_landed_avg',u'fighter_abs_head_strikes_attempts_avg',u'fighter_abs_body_strikes_landed_avg',u'fighter_abs_body_strikes_attempts_avg',u'fighter_abs_leg_strikes_landed_avg',u'fighter_abs_leg_strikes_attempts_avg',u'fighter_abs_distance_strikes_landed_avg',u'fighter_abs_distance_strikes_attempts_avg',u'fighter_abs_clinch_strikes_landed_avg',u'fighter_abs_clinch_strikes_attempts_avg',u'fighter_abs_ground_strikes_landed_avg',u'fighter_abs_ground_strikes_attempts_avg',
u'opponent_abs_knockdowns_avg',u'opponent_abs_pass_avg',u'opponent_abs_reversals_avg',u'opponent_abs_sub_attempts_avg',u'opponent_abs_takedowns_landed_avg',u'opponent_abs_takedowns_attempts_avg',u'opponent_abs_sig_strikes_landed_avg',u'opponent_abs_sig_strikes_attempts_avg',u'opponent_abs_total_strikes_landed_avg',u'opponent_abs_total_strikes_attempts_avg',u'opponent_abs_head_strikes_landed_avg',u'opponent_abs_head_strikes_attempts_avg',u'opponent_abs_body_strikes_landed_avg',u'opponent_abs_body_strikes_attempts_avg',u'opponent_abs_leg_strikes_landed_avg',u'opponent_abs_leg_strikes_attempts_avg',u'opponent_abs_distance_strikes_landed_avg',u'opponent_abs_distance_strikes_attempts_avg',u'opponent_abs_clinch_strikes_landed_avg',u'opponent_abs_clinch_strikes_attempts_avg',u'opponent_abs_ground_strikes_landed_avg',u'opponent_abs_ground_strikes_attempts_avg',
u'opponent_inf_knockdowns_avg',u'opponent_inf_pass_avg',u'opponent_inf_reversals_avg',u'opponent_inf_sub_attempts_avg',u'opponent_inf_takedowns_landed_avg',u'opponent_inf_takedowns_attempts_avg',u'opponent_inf_sig_strikes_landed_avg',u'opponent_inf_sig_strikes_attempts_avg',u'opponent_inf_total_strikes_landed_avg',u'opponent_inf_total_strikes_attempts_avg',u'opponent_inf_head_strikes_landed_avg',u'opponent_inf_head_strikes_attempts_avg',u'opponent_inf_body_strikes_landed_avg',u'opponent_inf_body_strikes_attempts_avg',u'opponent_inf_leg_strikes_landed_avg',u'opponent_inf_leg_strikes_attempts_avg',u'opponent_inf_distance_strikes_landed_avg',u'opponent_inf_distance_strikes_attempts_avg',u'opponent_inf_clinch_strikes_landed_avg',u'opponent_inf_clinch_strikes_attempts_avg',u'opponent_inf_ground_strikes_landed_avg',u'opponent_inf_ground_strikes_attempts_avg']

#adding record differences to ufc_fights
record_statistics_diff = []
half_length=int(len(record_statistics)/2)
for i in range(half_length):
    ufc_fights_winner[record_statistics[i]+'_diff_2']=ufc_fights_winner[record_statistics[i]]-ufc_fights_winner[record_statistics[i+half_length]]
    ufc_fights_method[record_statistics[i]+'_diff_2']=ufc_fights_method[record_statistics[i]]-ufc_fights_method[record_statistics[i+half_length]]
    record_statistics_diff.append(record_statistics[i]+'_diff_2')

#lets try and improve the greedy algorithm by considering differences. Lets start by replacing height and reach by their differences
ufc_fights_winner['height_diff']=ufc_fights_winner['fighter_height'].apply(float)-ufc_fights_winner['opponent_height'].apply(float)
ufc_fights_winner['reach_diff']=ufc_fights_winner['fighter_reach'].apply(float)-ufc_fights_winner['opponent_reach'].apply(float)
ufc_fights_method['height_diff']=ufc_fights_method['fighter_height'].apply(float)-ufc_fights_method['opponent_height'].apply(float)
ufc_fights_method['reach_diff']=ufc_fights_method['fighter_reach'].apply(float)-ufc_fights_method['opponent_reach'].apply(float)

physical_stats_diff = ['fighter_age_diff', 'height_diff', 'reach_diff']

#adding punch differences to ufc_fights
punch_statistics_diff = []
half_length=int(len(punch_statistics)/2)
for i in range(half_length):
    ufc_fights_method[punch_statistics[i]+'_diff_2']=ufc_fights_method[punch_statistics[i]]-ufc_fights_method[punch_statistics[i+half_length]]
    ufc_fights_winner[punch_statistics[i]+'_diff_2']=ufc_fights_winner[punch_statistics[i]]-ufc_fights_winner[punch_statistics[i+half_length]]
    punch_statistics_diff.append(punch_statistics[i]+'_diff_2')

possible_stats =record_statistics_diff+physical_stats_diff+punch_statistics_diff

#setting
ufc_fights_winner['fighter_age'] = ufc_fights_winner['fighter_age'].apply(float)
ufc_fights_winner['opponent_age'] = ufc_fights_winner['opponent_age'].apply(float)
ufc_fights_winner['fighter_age_diff'] = ufc_fights_winner['fighter_age']-ufc_fights_winner['opponent_age']

best_smallest_set = ['4-fighter_score_diff',
 '9-fighter_score_diff',
 '15-fighter_score_diff',
 '1-fight_math',
 '6-fight_math',
 'fighter_L5Y_sub_wins_diff_2',
 'fighter_L5Y_losses_diff_2',
 'fighter_L5Y_ko_losses_diff_2',
 'fighter_age_diff',
 'fighter_abs_total_strikes_landed_avg_diff_2',
 'fighter_abs_head_strikes_landed_avg_diff_2',
 'fighter_inf_ground_strikes_landed_avg_diff_2',
 'fighter_inf_takedowns_attempts_avg_diff_2',
 'fighter_inf_head_strikes_landed_avg_diff_2',
 ]

ufc_fights_df = ufc_fights_winner[best_smallest_set]

#decided to force intercept to be 0 due to symmetry of dataset (all stats are differences so if we switch fighters, we must get the negative of the result)
winPredictionModel=LogisticRegression(solver='lbfgs', max_iter=2000, fit_intercept=False)
X=ufc_fights_df.iloc[0:40*55].to_numpy()
y=ufc_fights_winner['result'].iloc[0:40*55]
print('Fitting Logistic Regression Model')
winPredictionModel.fit(X,y)
print('Accuracy of Logistic Regression win prediction: '+str(cross_val_score(winPredictionModel,X,y,cv=3).mean()))
print('coefficients'+str(winPredictionModel.coef_))
print('intercept'+str(winPredictionModel.intercept_))
theta = winPredictionModel.coef_
b = winPredictionModel.intercept_[0]


# We want to predict how many times out of 10 the winning fighter would win, so we look at the values
# x*theta+b. If the value is >=0 its a win and <=0 its a loss. But how far from zero gives indication of
# how likely the outcome is.

def presigmoid_value(fighter1,fighter2,date1,date2):
    value = 0
    tup = ufc_prediction_tuple(fighter1,fighter2,date1,date2)
    for i in range(len(tup)):
        value += tup[i]*theta[i]
    return value + b

def manual_prediction(fighter1,fighter2,date1,date2):
    value = presigmoid_value(fighter1,fighter2,date1,date2)
    value2 = presigmoid_value(fighter2,fighter1,date2,date1)
    return value-value2>=0

import math

def sigmoid(x):
    sig = 1 / (1 + math.exp(-x))
    return sig

#returns the probability that fighter1 defeats fighter2 on date1,date2
def probability(fighter1,fighter2,date1,date2):
    presig=presigmoid_value(fighter1,fighter2,date1,date2)
    return sigmoid(presig)

def odds(fighter1,fighter2):
    # custom date just for the purposes of fixing the prediction history after it was broken. Setting date to May 30
    date1 = 'May 30, 2023'
    date2 = 'May 30, 2023'
    p=probability(fighter1,fighter2,date1,date2)
    if p<.5:
        fighterOdds=round(100/p - 100)
        opponentOdds = round(1 / (1 / (1 - p) - 1) * 100)
        return ['+'+str(fighterOdds),'-'+str(opponentOdds)]
    elif p>=.5:
        fighterOdds = round(1 / (1 / p - 1) * 100)
        opponentOdds = round(100 / (1 - p) - 100)
        return ['-'+str(fighterOdds),'+'+str(opponentOdds)]

def give_odds(fighter1,fighter2,date1,date2):
    value = presigmoid_value(fighter1,fighter2,date1,date2)
    value2 = presigmoid_value(fighter2,fighter1,date2,date1)
    if value-value2>=0:
        winner=fighter1
    else:
        winner=fighter2
    value2 = presigmoid_value(fighter2,fighter1,date2,date1)
    abs_value = (abs(value)+abs(value2))/2
    if abs_value >=0 and abs_value <=.2:
        print(winner+" wins a little over 5 times out of 10 times.")
    elif abs_value >=.2 and abs_value <=.4:
        print(winner+" wins 6 out of 10 times.")
    elif abs_value >=.4 and abs_value <=.6:
        print(winner+" wins 7 out of 10 times.")
    elif abs_value >=.6 and abs_value <=.8:
        print(winner+" wins 9 out of 10 times.")
    elif abs_value >=.8:
        print(winner+" wins 10 out of 10 times.")

theta = list(theta[0])

theta_dict = {}
for i in range(len(theta)):
    theta_dict[i]=theta[i]

with open('content/data/external/theta.json', 'w') as outfile:
    json.dump(theta_dict, outfile)

intercept_dict = {0:b}

with open('content/data/external/intercept.json', 'w') as outfile:
    json.dump(intercept_dict, outfile)
    
print("Saved new theta and intercept to json files to run model in website")

#I've defined this in such a way to predict what happens when fighter1 in their day1 version fights fighter2
#in their day2 version. Meaning we could compare for example 2014 Tyron Woodley to 2019 Colby Covington
def ufc_prediction_tuple(fighter1,fighter2,day1=date.today(),day2=date.today()):
    return [fighter_score_diff(fighter1,fighter2,day1, 4),
            fighter_score_diff(fighter1,fighter2,day1, 9),
            fighter_score_diff(fighter1,fighter2,day1, 15),
            fight_math_diff(fighter1,fighter2,day1, 1),
            fight_math_diff(fighter1,fighter2,day1, 6),
            L5Y_sub_wins(fighter1,day1)-L5Y_sub_wins(fighter2,day2),
            L5Y_losses(fighter1,day1)-L5Y_losses(fighter2,day2),
            L5Y_ko_losses(fighter1,day1)-L5Y_ko_losses(fighter2,day2),
            fighter_age(fighter1,day1)-fighter_age(fighter2,day2),
            avg_count('total_strikes_landed',fighter1,'abs',day1)-avg_count('total_strikes_landed',fighter2,'abs',day2),
            avg_count('head_strikes_landed',fighter1,'abs',day1)-avg_count('head_strikes_landed',fighter2,'abs',day2),
            avg_count('ground_strikes_landed',fighter1,'inf',day1)-avg_count('ground_strikes_landed',fighter2,'inf',day2),
            avg_count('takedowns_attempts',fighter1,'inf',day1)-avg_count('takedowns_attempts',fighter2,'inf',day2),
            avg_count('head_strikes_landed',fighter1,'inf',day1)-avg_count('head_strikes_landed',fighter2,'inf',day2),
           ]

importing dataframe from ufc_fights.csv
Fitting Logistic Regression Model
Accuracy of Logistic Regression win prediction: 0.6295529426925541
coefficients[[ 0.02317681  0.00818608 -0.00395582  0.50810229  0.1277071   0.0442024
   0.04618844 -0.05344784 -0.07296933 -0.07810969 -0.02846047 -0.01266798
   0.5631782   0.07999932]]
intercept[0.]
Saved new theta and intercept to json files to run model in website


In [102]:
vegas_odds_col_names = list(prediction_history.columns)
vegas_odds_col_values = [['' for _ in range(new_fights.shape[0])] for _ in range(len(vegas_odds_col_names))]
vegas_odds_d = dict(zip(vegas_odds_col_names, vegas_odds_col_values))
vegas_odds = pd.DataFrame(data=vegas_odds_d)

vegas_odds['fighter name'].loc[:] = new_fights['fighter'].values
vegas_odds['opponent name'].loc[:] = new_fights['opponent'].values
vegas_odds['date'] = new_fights['date'].values

print('Making predictions for all fights on the books')
#filling in predictions
for i in vegas_odds.index:
    fighter=vegas_odds['fighter name'][i]
    opponent=vegas_odds['opponent name'][i]
    if in_ufc(fighter) and in_ufc(opponent):
        odds_calc = odds(fighter,opponent)
        print('predicting: '+fighter,'versus '+opponent,'.... '+str(odds_calc))
        vegas_odds['predicted fighter odds'][i]=odds_calc[0]
        vegas_odds['predicted opponent odds'][i]=odds_calc[1]
        sum_av_f=0
        tot_av_f=0
        sum_av_o=0
        tot_av_o=0
        for bookie in ['DraftKings', 'BetMGM', 'Caesars', 'BetRivers', 'FanDuel', 'PointsBet', 'Unibet', 'Bet365', 'BetWay', '5D', 'Ref']:
            if vegas_odds['fighter '+bookie][i]!='':
                sum_av_f+=int(vegas_odds['fighter '+bookie][i])
                tot_av_f+=1
            if vegas_odds['opponent '+bookie][i]!='':
                sum_av_o+=int(vegas_odds['opponent '+bookie][i])
                tot_av_o+=1
        if tot_av_f>0 and tot_av_o>0:
            vegas_odds['average bookie odds'][i]=[str(round(sum_av_f/tot_av_f)),str(round(sum_av_o/tot_av_o))]

Making predictions for all fights on the books
predicting: Alexander Volkanovski versus Yair Rodriguez .... ['-110', '+110']
predicting: Brandon Moreno versus Alexandre Pantoja .... ['-115', '+115']
predicting: Dricus Du Plessis versus Robert Whittaker .... ['-146', '+146']
predicting: Dan Hooker versus Jalin Turner .... ['+200', '-200']
predicting: Bo Nickal versus Val Woodburn .... ['-278', '+278']
predicting: Niko Price versus Robbie Lawler .... ['-189', '+189']
predicting: Edgar Chairez versus Tatsuro Taira .... ['+218', '-218']
predicting: Denise Gomes versus Yazmin Jauregui .... ['+117', '-117']
predicting: Alonzo Menifield versus Jimmy Crute .... ['+391', '-391']
predicting: Vitor Petrino versus Marcin Prachnio .... ['-259', '+259']
predicting: Terrence Mitchell versus Cameron Saaiman .... ['+252', '-252']
predicting: Jesus Aguilar versus Shannon Ross .... ['-737', '+737']
predicting: Esteban Ribovics versus Kamuela Kirk .... ['-121', '+121']
predicting: Sean Strickland versus A

In [104]:
vegas_odds_old = vegas_odds.copy()

In [105]:
vegas_odds_old

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref,predicted fighter odds,predicted opponent odds,average bookie odds,correct?,date
0,Alexander Volkanovski,,,,,,,,,,,,Yair Rodriguez,,,,,,,,,,,,-110,+110,,,"July 08, 2023"
1,Brandon Moreno,,,,,,,,,,,,Alexandre Pantoja,,,,,,,,,,,,-115,+115,,,"July 08, 2023"
2,Dricus Du Plessis,,,,,,,,,,,,Robert Whittaker,,,,,,,,,,,,-146,+146,,,"July 08, 2023"
3,Dan Hooker,,,,,,,,,,,,Jalin Turner,,,,,,,,,,,,+200,-200,,,"July 08, 2023"
4,Bo Nickal,,,,,,,,,,,,Val Woodburn,,,,,,,,,,,,-278,+278,,,"July 08, 2023"
5,Niko Price,,,,,,,,,,,,Robbie Lawler,,,,,,,,,,,,-189,+189,,,"July 08, 2023"
6,Edgar Chairez,,,,,,,,,,,,Tatsuro Taira,,,,,,,,,,,,+218,-218,,,"July 08, 2023"
7,Denise Gomes,,,,,,,,,,,,Yazmin Jauregui,,,,,,,,,,,,+117,-117,,,"July 08, 2023"
8,Alonzo Menifield,,,,,,,,,,,,Jimmy Crute,,,,,,,,,,,,+391,-391,,,"July 08, 2023"
9,Vitor Petrino,,,,,,,,,,,,Marcin Prachnio,,,,,,,,,,,,-259,+259,,,"July 08, 2023"


In [108]:
# getting rid of fights that didn't actually happen and adding correctness results of those that did
bad_indices = []
for index1, row1 in vegas_odds_old.iterrows():
    card_date = row1['date']
    relevant_fights = ufc_fights_crap
    match_found = False
    fighter_odds = row1['predicted fighter odds']
    if fighter_odds == '':
        bad_indices.append(index1)
    else:
        for index2, row2 in relevant_fights.iterrows():
            if same_name(row1['fighter name'], row2['fighter']) and same_name(row1['opponent name'], row2['opponent']):
                match_found = True
                print('adding fight from '+str(card_date)+' between '+row1['fighter name']+' and '+row1['opponent name'])
                if (int(fighter_odds) < 0 and row2['result'] == 'W') or (int(fighter_odds) > 0 and row2['result'] == 'L'):
                    vegas_odds_old.at[index1,'correct?'] = 1
                else:
                    vegas_odds_old.at[index1,'correct?'] = 0
                break
        if not match_found:
            bad_indices.append(index1)
            print('fight from '+card_date+' between '+row1['fighter name']+' and '+row1['opponent name'] + ' not found in ufc_fights_crap.csv')
# drop bad indices

adding fight from July 08, 2023 between Alexander Volkanovski and Yair Rodriguez
adding fight from July 08, 2023 between Brandon Moreno and Alexandre Pantoja
adding fight from July 08, 2023 between Dricus Du Plessis and Robert Whittaker
adding fight from July 08, 2023 between Dan Hooker and Jalin Turner
adding fight from July 08, 2023 between Bo Nickal and Val Woodburn
adding fight from July 08, 2023 between Niko Price and Robbie Lawler
adding fight from July 08, 2023 between Edgar Chairez and Tatsuro Taira
adding fight from July 08, 2023 between Denise Gomes and Yazmin Jauregui
adding fight from July 08, 2023 between Alonzo Menifield and Jimmy Crute
adding fight from July 08, 2023 between Vitor Petrino and Marcin Prachnio
adding fight from July 08, 2023 between Terrence Mitchell and Cameron Saaiman
adding fight from July 08, 2023 between Jesus Aguilar and Shannon Ross
adding fight from July 08, 2023 between Esteban Ribovics and Kamuela Kirk
adding fight from July 01, 2023 between Sean

In [109]:
vegas_odds_old = vegas_odds_old.drop(bad_indices)

In [110]:
vegas_odds_old

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref,predicted fighter odds,predicted opponent odds,average bookie odds,correct?,date
0,Alexander Volkanovski,,,,,,,,,,,,Yair Rodriguez,,,,,,,,,,,,-110,+110,,1,"July 08, 2023"
1,Brandon Moreno,,,,,,,,,,,,Alexandre Pantoja,,,,,,,,,,,,-115,+115,,0,"July 08, 2023"
2,Dricus Du Plessis,,,,,,,,,,,,Robert Whittaker,,,,,,,,,,,,-146,+146,,1,"July 08, 2023"
3,Dan Hooker,,,,,,,,,,,,Jalin Turner,,,,,,,,,,,,+200,-200,,0,"July 08, 2023"
4,Bo Nickal,,,,,,,,,,,,Val Woodburn,,,,,,,,,,,,-278,+278,,1,"July 08, 2023"
5,Niko Price,,,,,,,,,,,,Robbie Lawler,,,,,,,,,,,,-189,+189,,0,"July 08, 2023"
6,Edgar Chairez,,,,,,,,,,,,Tatsuro Taira,,,,,,,,,,,,+218,-218,,1,"July 08, 2023"
7,Denise Gomes,,,,,,,,,,,,Yazmin Jauregui,,,,,,,,,,,,+117,-117,,0,"July 08, 2023"
8,Alonzo Menifield,,,,,,,,,,,,Jimmy Crute,,,,,,,,,,,,+391,-391,,0,"July 08, 2023"
9,Vitor Petrino,,,,,,,,,,,,Marcin Prachnio,,,,,,,,,,,,-259,+259,,1,"July 08, 2023"


In [111]:
#making a copy of vegas_odds
vegas_odds_copy=vegas_odds_old.copy()
prediction_history=pd.read_json('content/data/external/prediction_history.json')
#for col in ['predicted fighter odds','predicted opponent odds','average bookie odds','correct?']:
#    vegas_odds_copy[col]=""

#add the newly scraped fights and predicted fights to the history of prediction list (idea: might be better to wait to join until after the fights happen)
prediction_history = pd.concat([vegas_odds_copy, prediction_history], axis = 0).reset_index(drop=True)

In [112]:
prediction_history

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref,predicted fighter odds,predicted opponent odds,average bookie odds,correct?,date
0,Alexander Volkanovski,,,,,,,,,,,,Yair Rodriguez,,,,,,,,,,,,-110,+110,,1,"July 08, 2023"
1,Brandon Moreno,,,,,,,,,,,,Alexandre Pantoja,,,,,,,,,,,,-115,+115,,0,"July 08, 2023"
2,Dricus Du Plessis,,,,,,,,,,,,Robert Whittaker,,,,,,,,,,,,-146,+146,,1,"July 08, 2023"
3,Dan Hooker,,,,,,,,,,,,Jalin Turner,,,,,,,,,,,,+200,-200,,0,"July 08, 2023"
4,Bo Nickal,,,,,,,,,,,,Val Woodburn,,,,,,,,,,,,-278,+278,,1,"July 08, 2023"
5,Niko Price,,,,,,,,,,,,Robbie Lawler,,,,,,,,,,,,-189,+189,,0,"July 08, 2023"
6,Edgar Chairez,,,,,,,,,,,,Tatsuro Taira,,,,,,,,,,,,+218,-218,,1,"July 08, 2023"
7,Denise Gomes,,,,,,,,,,,,Yazmin Jauregui,,,,,,,,,,,,+117,-117,,0,"July 08, 2023"
8,Alonzo Menifield,,,,,,,,,,,,Jimmy Crute,,,,,,,,,,,,+391,-391,,0,"July 08, 2023"
9,Vitor Petrino,,,,,,,,,,,,Marcin Prachnio,,,,,,,,,,,,-259,+259,,1,"July 08, 2023"


In [113]:
#saving the new prediction_history dataframe to json
result = prediction_history.to_json()
parsed = json.loads(result)
jsonFilePath='content/data/external/prediction_history.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)

saved to content/data/external/prediction_history.json


In [114]:
vegas_odds = pd.read_json('content/data/external/vegas_odds.json')

In [116]:
vegas_odds_old

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref,predicted fighter odds,predicted opponent odds,average bookie odds,correct?,date
0,Alexander Volkanovski,,,,,,,,,,,,Yair Rodriguez,,,,,,,,,,,,-110,+110,,1,"July 08, 2023"
1,Brandon Moreno,,,,,,,,,,,,Alexandre Pantoja,,,,,,,,,,,,-115,+115,,0,"July 08, 2023"
2,Dricus Du Plessis,,,,,,,,,,,,Robert Whittaker,,,,,,,,,,,,-146,+146,,1,"July 08, 2023"
3,Dan Hooker,,,,,,,,,,,,Jalin Turner,,,,,,,,,,,,+200,-200,,0,"July 08, 2023"
4,Bo Nickal,,,,,,,,,,,,Val Woodburn,,,,,,,,,,,,-278,+278,,1,"July 08, 2023"
5,Niko Price,,,,,,,,,,,,Robbie Lawler,,,,,,,,,,,,-189,+189,,0,"July 08, 2023"
6,Edgar Chairez,,,,,,,,,,,,Tatsuro Taira,,,,,,,,,,,,+218,-218,,1,"July 08, 2023"
7,Denise Gomes,,,,,,,,,,,,Yazmin Jauregui,,,,,,,,,,,,+117,-117,,0,"July 08, 2023"
8,Alonzo Menifield,,,,,,,,,,,,Jimmy Crute,,,,,,,,,,,,+391,-391,,0,"July 08, 2023"
9,Vitor Petrino,,,,,,,,,,,,Marcin Prachnio,,,,,,,,,,,,-259,+259,,1,"July 08, 2023"


In [123]:
def get_next_fight_card():
    url = 'https://www.bestfightodds.com'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser") 
    mycards = soup.find_all("div", {"class": "table-div"})
    ufc_cards = [card for card in mycards if 'ufc' in card.find("a",href=lambda x: x and x.startswith('/events')).get_text().lower()]
    card = ufc_cards[0]
    card_date = card.find("span",{"class":"table-header-date"}).get_text()
    card_title = card.find("a",href=lambda x: x and x.startswith('/events')).get_text()
    #bookies = [bookie.get_text() for bookie in card.find_all("a",href=lambda x: x and x.startswith('/out'))]
    fighter_divs = card.find_all('span',{"class":"t-b-fcc"})
    fighters_list = [fighter.get_text() for fighter in fighter_divs]
    if len(fighters_list)%2 == 0 and all(fighters_list[i] == fighters_list[i+26] for i in range(len(fighters_list)//2)):
        print('Usual Scraping Structure Detected')
    else:
        print('Scraping Structure Has Changed... Check HTML at BestFightOdds.com')
    fights_list = [fighters_list[i:i+2] for i in range(0,len(fighters_list),2)]
    fights_list = fights_list[:len(fights_list)//2]
    print(f'Upcoming card {card_title} on {card_date} has {len(fights_list)} fights')
    for fight in fights_list:
        print(fight)
    return card_date, card_title, fights_list

In [124]:
card_date, card_title, fights_list = get_next_fight_card()
card_date = convert_scraped_date_to_standard_date(card_date)

card_info_dict = {"date":card_date, "title":card_title}

print('Writing upcoming card info to content/data/external/card_info.json')
with open('content/data/external/card_info.json', 'w') as outfile:
    json.dump(card_info_dict, outfile)
print("###############################################################################################################")


#now we build prediction_history
#next open existing prediction_history.json as a dataframe (done)... then add all fights on the books from vegas_odds
#for each fight in prediction_history, if a prediction has not yet been made, make a prediction... then export to json

Usual Scraping Structure Detected
Upcoming card UFC Fight Night 225 on July 15th has 13 fights
['Holly Holm', 'Mayra Bueno Silva']
['Bassil Hafez', 'Jack Della Maddalena']
['Francisco Prado', 'Ottman Azaitar']
['Albert Duraev', 'Jun Yong Park']
['Chelsea Chandler', 'Norma Dumont']
['Nazim Sadykhov', 'Terrance McKinney']
['Melsik Baghdasaryan', 'Tucker Lutz']
['Istela Nunes', 'Viktoriia Dudakova']
['Austin Lingo', 'Melquizael Costa']
['Evan Elder', 'Genaro Valdez']
['Azat Maksum', 'Tyson Nam']
['Alexander Munoz', 'Carl Deaton']
['Ailin Perez', 'Ashlee Evans-Smith']
Writing upcoming card info to content/data/external/card_info.json
###############################################################################################################


In [125]:
vegas_odds_col_names = list(prediction_history.columns)
vegas_odds_col_values = [['' for _ in range(len(fights_list))] for _ in range(len(vegas_odds_col_names))]
vegas_odds_d = dict(zip(vegas_odds_col_names, vegas_odds_col_values))
vegas_odds = pd.DataFrame(data=vegas_odds_d)

vegas_odds['fighter name'].loc[:] = [fight[0] for fight in fights_list]
vegas_odds['opponent name'].loc[:] = [fight[1] for fight in fights_list]
vegas_odds['date'] = card_date

In [126]:
print('Making predictions for all fights on the books')
#filling in predictions
for i in vegas_odds.index:
    fighter=vegas_odds['fighter name'][i]
    opponent=vegas_odds['opponent name'][i]
    if in_ufc(fighter) and in_ufc(opponent):
        odds_calc = odds(fighter,opponent)
        print('predicting: '+fighter,'versus '+opponent,'.... '+str(odds_calc))
        vegas_odds['predicted fighter odds'][i]=odds_calc[0]
        vegas_odds['predicted opponent odds'][i]=odds_calc[1]
        sum_av_f=0
        tot_av_f=0
        sum_av_o=0
        tot_av_o=0
        for bookie in ['DraftKings', 'BetMGM', 'Caesars', 'BetRivers', 'FanDuel', 'PointsBet', 'Unibet', 'Bet365', 'BetWay', '5D', 'Ref']:
            if vegas_odds['fighter '+bookie][i]!='':
                sum_av_f+=int(vegas_odds['fighter '+bookie][i])
                tot_av_f+=1
            if vegas_odds['opponent '+bookie][i]!='':
                sum_av_o+=int(vegas_odds['opponent '+bookie][i])
                tot_av_o+=1
        if tot_av_f>0 and tot_av_o>0:
            vegas_odds['average bookie odds'][i]=[str(round(sum_av_f/tot_av_f)),str(round(sum_av_o/tot_av_o))]
            
print('saving scraped fights and predictions to content/data/external/vegas_odds.json')
print('TODO: scrape odds too. Currently only scraping names, date, and card title')
#save to json
result = vegas_odds.to_json()
parsed = json.loads(result)
jsonFilePath='content/data/external/vegas_odds.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)

Making predictions for all fights on the books
predicting: Holly Holm versus Mayra Bueno Silva .... ['+181', '-181']
predicting: Francisco Prado versus Ottman Azaitar .... ['-154', '+154']
predicting: Albert Duraev versus Jun Yong Park .... ['+121', '-121']
predicting: Chelsea Chandler versus Norma Dumont .... ['-119', '+119']
predicting: Nazim Sadykhov versus Terrance McKinney .... ['+194', '-194']
predicting: Melsik Baghdasaryan versus Tucker Lutz .... ['+100', '-100']
predicting: Austin Lingo versus Melquizael Costa .... ['-115', '+115']
predicting: Evan Elder versus Genaro Valdez .... ['-170', '+170']
predicting: Alexander Munoz versus Carl Deaton .... ['-113', '+113']
predicting: Ailin Perez versus Ashlee Evans-Smith .... ['-156', '+156']
saving scraped fights and predictions to content/data/external/vegas_odds.json
TODO: scrape odds too. Currently only scraping names, date, and card title
saved to content/data/external/vegas_odds.json
